In [15]:
import dedupe
import json
from cytoolz import *
from copy import copy
import re
import nltk
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
from urllib.parse import urlparse
from unicodedata import normalize

In [16]:
fields = [{'field': 'description', 'type': 'String', 'has missing': True},
          {'field': 'description', 'type': 'Text', 'has missing': True},
          {'field': 'price', 'type': 'Price', 'has missing': True}]


def tokenize_url(url):
    path = urlparse(url).path
    path = path.replace('/', ' ')
    path = path.replace('-', ' ')
    return word_tokenize(path)


def remove_accents(input_str):
    return normalize('NFKD', input_str).encode('ascii', 'ignore').decode()


def sanitize_string(input_str):
    pattern = r'[?|&|!|@|#|;|*|~|(|)|´|^|\r|\n|\t]'
    return clean_string(re.sub(pattern, r'', remove_accents(input_str.lower())))


def clean_string(input_str):
    unuseful = set(['confira', 'compre', 'oferta', 'novo',
                    'preco', 'agora', 'melhores', 'aqui',
                    'aproveite', 'menor', 'maior', 'encontra',
                    'site', 'condicoes', 'ofertas', 'imbativeis',
                    'vendas', 'online', 'novo', 'nova', 'tecnologia',
                    'precos', 'pagamento', 'melhor', 'veja', 'encontre',
                    'menores', 'vem', 'venha', 'ver'])

    names = set(['walmart', 'walmart.com', 'shoptime', 'shoptime.com',
                 'pontofrio', 'pontofrio.com', 'magazineluiza', 'magazineluiza.com',
                 'magazine luiza', 'americanas', 'americanas.com', 'submarino',
                 'submarino.com', 'extra', 'extra.com.br', 'casas bahia', 'casasbahia.com.br'])

    stopwords = [remove_accents(s) for s in nltk.corpus.stopwords.words('portuguese')]
    repl = list(concat([unuseful, names, stopwords]))
    tk = word_tokenize(input_str)
    return ' '.join([word for word in tk if word not in repl])


def preprocess(item):
    return {'description': sanitize_string(item['description']), 'price': float(sanitize_string(item['price']))}

In [17]:
with open('../data/americanas/buscape.json') as f:
    walmart = [preprocess(dissoc(item, 'image')) for item in json.loads(f.read())['products']]

with open('../data/magazineluiza/buscape.json') as f:
    submarino = [preprocess(dissoc(item, 'image')) for item in json.loads(f.read())['products']]

In [18]:
p = {str(i): walmart[i] for i in range(len(walmart))}
w = {str(i): submarino[i] for i in range(len(submarino))}

In [19]:
linker = dedupe.RecordLink(fields)
linker.sample(p, w)
dedupe.consoleLabel(linker)

description : kit reparo piscina 106400 . 0824175
price : 14.9

description : kit reparo balanca traseira peugeot 206 207 1.4 1.6 todos barra estabilizadora
price : 105.9

0/10 positive, 0/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished


n


description : tapete aviao - castor 8372374
price : 59.9

description : tapete personalizado nissan x - terra 2009 2012 preto 5 pcs + trava seguranca
price : 59.9

0/10 positive, 1/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


n


description : escova dental colgate smiles 5+ anos 8306039
price : 17.19

description : escova dental colgate smiles 5 anos
price : 14.99

0/10 positive, 2/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


...

description : guarda roupa casal 6 portas 3 gavetas caju henn malbec 8270409
price : 579.9

description : guarda roupa casal 6 portas 8 gavetas espelho coliseu moveis horizonte freijo dourado
price : 1899.9

11/10 positive, 40/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


n


description : guarda - roupa casal athenas 6 portas 4 gavetas 16554 . 1234973
price : 2043.0

description : guarda roupa infantil rubi 4 portas - canaa
price : 1195.2

11/10 positive, 41/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished / (p)revious


f


Finished labeling


In [20]:
linker.train()

INFO:rlr.crossvalidation:using cross validation to find optimum alpha...
INFO:rlr.crossvalidation:optimum alpha: 0.100000
INFO:dedupe.blocking:Canopy: TfidfNGramSearchPredicate: (0.8, description)
INFO:dedupe.blocking:Canopy: TfidfNGramSearchPredicate: (0.4, description)
INFO:dedupe.blocking:Canopy: TfidfNGramSearchPredicate: (0.2, description)
INFO:dedupe.blocking:Canopy: TfidfNGramSearchPredicate: (0.6, description)
INFO:dedupe.blocking:Canopy: LevenshteinSearchPredicate: (1, description)
INFO:dedupe.blocking:Canopy: LevenshteinSearchPredicate: (2, description)
INFO:dedupe.blocking:Canopy: LevenshteinSearchPredicate: (3, description)
INFO:dedupe.blocking:Canopy: LevenshteinSearchPredicate: (4, description)
INFO:dedupe.blocking:Canopy: TfidfTextSearchPredicate: (0.8, description)
INFO:dedupe.blocking:Canopy: TfidfTextSearchPredicate: (0.6, description)
INFO:dedupe.blocking:Canopy: TfidfTextSearchPredicate: (0.2, description)
INFO:dedupe.blocking:Canopy: TfidfTextSearchPredicate: (0.4,

In [24]:
m = linker.match(p,w, 0.2)

INFO:dedupe.blocking:Canopy: TfidfNGramSearchPredicate: (0.6, description)
INFO:dedupe.blocking:Canopy: TfidfTextSearchPredicate: (0.6, description)
INFO:dedupe.api:0 records
INFO:dedupe.api:100 records


In [25]:
m

[(('8700', '2351'), 0.91932768),
 (('8706', '2353'), 0.81043434),
 (('8707', '2364'), 0.81043434),
 (('8576', '3940'), 0.61180764),
 (('8578', '3942'), 0.57256919),
 (('3564', '6227'), 0.50662416),
 (('8577', '3945'), 0.47610179),
 (('2943', '3863'), 0.46175286),
 (('2028', '6945'), 0.4521119),
 (('8703', '2356'), 0.43386456),
 (('7308', '466'), 0.43194285),
 (('2032', '6943'), 0.35962871),
 (('8130', '802'), 0.26454091),
 (('3565', '6226'), 0.23791718),
 (('3571', '6212'), 0.21998759)]

In [26]:
for i in m:
    print(p[i[0][0]], w[i[0][1]])

{'description': 'escova dental colgate smiles 5 anos', 'price': 14.99} {'description': 'escova dental colgate smiles 5+ anos 8306039', 'price': 17.19}
{'description': 'escova dental close up ultra reach media', 'price': 14.44} {'description': 'escova dental closeup ultra reach media 1 unidade 8253885', 'price': 16.47}
{'description': 'escova dental close up ultra reach macia', 'price': 14.44} {'description': 'escova dental closeup ultra reach macia 1 unidade 8207840', 'price': 16.47}
{'description': 'calca motocross pro tork insane 4 vermelha cinza trilha', 'price': 106.9} {'description': 'calca motocross pro tork insane 4 amarela roxa trilha - tamanho 38 8522268', 'price': 126.88}
{'description': 'calca motocross pro tork insane 3 azul trilha', 'price': 99.9} {'description': 'calca motocross pro tork insane 4 amarela roxa trilha - tamanho 40 8535169', 'price': 126.88}
{'description': 'guarda - roupa infantil selena 4 portas 2 gavetas branco - carolina baby', 'price': 1014.9} {'descrip

In [5]:
len(walmart)

9035

In [6]:
len(submarino)

8970